# Defining Function Calling

## OpenAI's tools

* Can be used to return more specific information
* Can define a more precise structure
* Enhances the capabilities of the API call


Example:
Creating a function to interact with a smart home device

- We need the response be consistent across different users, so the hardware gets the correct instruction.

In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
import os

api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(
    api_key=api_key
)

In [2]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'user',
            'content': 'Please write down four trees with their scientific names, only return the list in json without any extra enclosing character'
        }
    ],
    response_format={'type': 'json_object'}
)

In [3]:
print(response.choices[0].message.content)

{
  "trees": [
    {
      "common_name": "Oak",
      "scientific_name": "Quercus"
    },
    {
      "common_name": "Maple",
      "scientific_name": "Acer"
    },
    {
      "common_name": "Pine",
      "scientific_name": "Pinus"
    },
    {
      "common_name": "Birch",
      "scientific_name": "Betula"
    }
  ]
}


> At this point we have no warranty to get exactly the format we need among all the users

That's why the Function Callings are here to help


## Use cases for function calling

* Going from unstructured to consistent structured output
* Calling multiple functions to provide complex responses
* Calling external APIs



## Configure Function Calling

Highlights
* `tools` parameter
* function definition, the way we need to define the function

In [4]:
## Extracting structured data from text

function_definition = [{'type': 'function',
                        'function': {'name': 'real_estate_info',
                                     'description': 'Get the information about homes for sale from the body of the input text',
                                     'parameters': {'type': 'object',
                                                    'properties': {
                                                        'home type': {'type': 'string', 'description': 'Home type'},
                                                        'location': {'type': 'string', 'description': 'Location'},
                                                        'price': {'type': 'integer', 'description': 'Price'},
                                                        'bedrooms': {'type': 'integer',
                                                                     'description': 'Number of bedrooms'}}}}}]

message_listing = [{'role': 'system',
                    'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
                   {'role': 'user',
                    'content': 'Step into this beautiful two-story, single-family home located in Springfield, USA, priced at $350,000. This charming property features 4 bedrooms, 2.5 bathrooms, a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances, and a large backyard perfect for family gatherings. The master bedroom includes an en-suite bathroom and a walk-in closet. Enjoy the convenience of an attached two-car garage and a recently updated HVAC system. Located near top-rated schools, parks, and shopping centers, this home is ideal for families looking for comfort and convenience.'}]

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message_listing,
    tools=function_definition
)


In [5]:
print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"home type":"single-family","location":"Springfield, USA","price":350000,"bedrooms":4}', name='real_estate_info')


In [6]:
print(response.choices[0].message.tool_calls[0].function.arguments)

{"home type":"single-family","location":"Springfield, USA","price":350000,"bedrooms":4}


## Working with Multiple Functions

> **Parallel Function Calling** : The ability of the model to handle multiple functions.

* Passing multiple functions to be used in the response
* **Default behavior is to choose which one to use depending on the input**


In [14]:
schools_check = {
    "type": "function",
    "function": {
        "name": "schools",
        "description": "Review if there are schools nearby",
        "parameters": {
            "type": "object",
            "properties": {
                "school_nearby": {
                    "type": "boolean",
                    "description": "School nearby"
                }
            },
            "required": ["school_nearby"]
        }
    }
}

function_definition.append(schools_check)


In [8]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message_listing,
    tools=function_definition
)

In [9]:
print(response.choices[0].message.tool_calls[0].function.arguments)

{"home type": "single-family", "location": "Springfield, USA", "price": 350000, "bedrooms": 4}


In [10]:
print(response.choices[0].message.tool_calls[1].function.arguments)

{"school_nearby": true}


### Function calling selection

* The function can be executed or not by default
    * `tool_choice`=`auto`
* Make a function call execution mandatory
    *  `tool_choice`={
       'type' = 'function',
            'function': {'name':'function_name'}
    }



In [13]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=message_listing,
    tools=function_definition,
    tool_choice={'type': 'function',
                 'function': {'name': 'schools'}
    }
)
print(response.choices[0].message.tool_calls[0].function.arguments)

{"school_nearby":true}


### Unexpected responses

The model can return a function response even if there is no information

To prevent this behavior

```
{"role":"system","content":"Don't make assumptions about what values to plug into functions. Don't make up values to fill the response with."}
{"role":"system","content":"Ask for clarification if needed."}
{"role":"user","content":"What is the starting salary for the role?."}
```

### The result may be empty

If there are no data to run the functions then the response is empty

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=message_listing,
    tools=function_definition
)

print(response.choices[0].message.content)

> Answer: I don't have information about the starting salary for the role. If you'd like, I can help you extract the job information from the description and then provide additional assistance.
